### Importación de Bibliotecas

In [20]:
import pandas as pd
import os
import ast

### Cargar el conjunto de datos limpios

In [21]:
df_steam_games = pd.read_csv('steam_games_cleaned.csv')
df_user_reviews = pd.read_csv('user_reviews_cleaned.csv')
df_user_items = pd.read_csv('user_items_cleaned.csv')

In [24]:
def UserForGenre(genero: str):
    # Merge de los DataFrames
    df_merged = pd.merge(df_user_items, df_steam_games[['item_id', 'genres', 'year']], on='item_id', how='inner')

    # Convertir la columna 'genres' a listas
    df_merged['genres'] = df_merged['genres'].apply(eval)

    # Filtrar el DataFrame para el género específico
    filtered_df = df_merged[df_merged['genres'].apply(lambda x: genero in x)]

    # Agrupar por usuario y año de lanzamiento y sumar las horas jugadas
    grouped_df = filtered_df.groupby(['user_id', 'year'])['hours_game'].sum().reset_index()

    if grouped_df.empty:
        return {"mensaje": f"No hay datos disponibles para el género {genero}."}

    # Encontrar el usuario con más horas jugadas para el género dado
    max_hours_user = grouped_df.groupby('user_id')['hours_game'].sum().idxmax()

    # Filtrar el DataFrame para el usuario con más horas jugadas
    user_df = grouped_df[grouped_df['user_id'] == max_hours_user]

    # Construir la lista de la acumulación de horas jugadas por año
    horas_por_anio = [{"Año": int(row['year']), "Horas": row['hours_game']} for _, row in user_df.iterrows()]

    # Construir el response_data
    response_data = {
        f"Usuario con más horas jugadas para {genero}": max_hours_user,
        "Horas jugadas": horas_por_anio
    }

    # Mostrar el resultado
    return response_data


In [25]:
UserForGenre('Action')

TypeError: eval() arg 1 must be a string, bytes or code object

### Rutinas

##### - 1. PlayTimeGenre

In [4]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_items, df_steam_games[['item_id', 'year', 'genres']], on='item_id', how='inner')

#Agrupamos por genero y año de lanzamiento y sumar las horas jugadas
#df_merged = df_merged.groupby(['genres', 'year']).agg({'hours_game': 'sum'}).reset_index()

#Agrupamos por genero y año de lanzamiento
df_merged = df_merged.groupby(['year', 'genres'])['hours_game'].sum().reset_index()

#Convertimos la columna genres a tipo lista
df_merged['genres'] = df_merged['genres'].apply(ast.literal_eval)

#Expandimos las listas en columnas
df_merged = df_merged.explode('genres').reset_index(drop=True)

# Encontrar el género con más horas jugadas por año 
max_hours_genre = df_merged.loc[df_merged.groupby(['year', 'genres'])['hours_game'].idxmax()]

# Ordenar el DataFrame resultante
max_hours_genre = max_hours_genre.sort_values(by=['genres', 'hours_game', 'year'], ascending=[True, False, True])

# Guardar el resultado en un archivo CSV
max_hours_genre.to_csv('PlayTimeGenre.csv', index=False)

##### - 3. UsersRecommend

In [4]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año
top3_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_by_year.to_csv('UsersRecommend.csv', index=False)

##### - 4. UsersWorstDeveloper

In [5]:
# Unir los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
 
# Renombrar la columna 'year_x' a 'year'
df_filtered = df_filtered.rename(columns={'year_x': 'year'})

# Contar las ocurrencias de cada desarrolladora por año
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
    
# Ordenar por 'year' y 'count' en orden descendente
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    

# Obtener el top 3 por año
result_df = grouped_result.groupby('year').head(3)    

# Guardar el resultado en un archivo CSV
result_df.to_csv('UsersWorstDeveloper.csv', index=False)

##### - 5. sentiment_analysis

In [6]:
# Merge de los DataFrames
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_csv('sentiment_analysis.csv', index=False)